In [1]:
# Standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Handeling .wav files

import librosa
from librosa import feature

# Machine Learning

from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from functools import reduce

# data vizualisation

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    
    # Add the number of dB to each column name for each .csv file

    df_6dB.columns = [str(col) + '_6dB' for col in df_6dB.columns]
    df_0dB.columns = [str(col) + '_0dB' for col in df_0dB.columns]
    df_min6dB.columns = [str(col) + '_-6dB' for col in df_min6dB.columns]
    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = reduce(lambda  left,right: pd.merge(left,right,left_index=True, right_index=True,how='outer'), data_frames)
    
    df_merged.head()
    
    return df_merged

In [3]:
valve = merge_pd('valve')

In [4]:
valve.drop(columns=['normal(0)/abnormal(1)_6dB','normal(0)/abnormal(1)_0dB'],axis=1,inplace=True)
valve.rename(columns={'normal(0)/abnormal(1)_-6dB': 'normal(0)/abnormal(1)'}, inplace=True)

In [5]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

In [6]:
# 20% of the overal data will seperated for later validation of the model

X_model, X_valid = train_test_split(valve,test_size=0.2,random_state = 42)

y_valid = X_valid['normal(0)/abnormal(1)']
X_valid = X_valid.drop(columns=['normal(0)/abnormal(1)'])

# 64% of the overal data (80% of X_model, y_model) will be used to create a training set for the model
# 16% of the overal data (20% of X_model, y_model) will be used to create a testing set for the model

X_train, X_test, y_train, y_test = train_test_split(X_model.drop(columns=['normal(0)/abnormal(1)'], axis=1),
                                                   X_model['normal(0)/abnormal(1)'],
                                                   test_size = 0.25,
                                                   random_state = 10)

In [7]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 56em; }</style>"))

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import RepeatedStratifiedKFold

y_pred = []
clfs = []

clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(RandomForestClassifier(class_weight='balanced'))
#clfs.append(EasyEnsembleClassifier(importance_getter=index))

#feature_list = [6,13,21]

#for index in feature_list:
for index in range(4,22):
    
    for classifier in clfs:
        
        print("----------------------------------------------")
        print("----------------------------------------------")
        print(classifier)
        print("----------------------------------------------")
        print("----------------------------------------------")
        
        sel = RFE(classifier, n_features_to_select = index)
        sel.fit(X_train, y_train)
        features = X_train.columns[sel.get_support()]
        X_train_rfe = sel.transform(X_train)
        X_test_rfe = sel.transform(X_test)
        print('Selected Feature', index)
        print(features)
    
        classifier.fit(X_train_rfe, y_train)
        y_pred= classifier.predict(X_test_rfe)
        scores = cross_val_score(pipeline, X_train_rfe, y_train, cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1))



        print("----------------------------------------------")
        print("TRAIN-TEST")
        print("----------------------------------------------")


        print('confusion matrix', classifier)
        print(confusion_matrix(y_test, y_pred))
        print('classification report')
        print(classification_report(y_test, y_pred))
        print('accuracy score')
        print(accuracy_score(y_test, y_pred))
        
        X_valid_rfe = sel.transform(X_valid)
        y_pred = classifier.predict(X_valid_rfe)

        print("----------------------------------------------")
        print("TRAIN-VALIDATION")
        print("----------------------------------------------")

        print('confusion matrix', classifier)
        print(confusion_matrix(y_valid, y_pred))
        print('classification report')
        print(classification_report(y_valid, y_pred))
        print('accuracy score')
        print(accuracy_score(y_valid, y_pred))

----------------------------------------------
----------------------------------------------
DecisionTreeClassifier()
----------------------------------------------
----------------------------------------------
Selected Feature 4
Index(['melspectrogram_max_6dB', 'melspectrogram_std_6dB', 'max perc_6dB',
       'max perc_0dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[699  35]
 [ 30  70]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       734
           1       0.67      0.70      0.68       100

    accuracy                           0.92       834
   macro avg       0.81      0.83      0.82       834
weighted avg       0.92      0.92      0.92       834

accuracy score
0.9220623501199041
----------------------------------------------
TRAIN-VALIDATION
------------------------------

Selected Feature 6
Index(['melspectrogram_max_6dB', 'melspectrogram_corr_6dB', 'max perc_6dB',
       'max perc_0dB', 'min perc_0dB', 'max perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[713  21]
 [ 30  70]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       734
           1       0.77      0.70      0.73       100

    accuracy                           0.94       834
   macro avg       0.86      0.84      0.85       834
weighted avg       0.94      0.94      0.94       834

accuracy score
0.9388489208633094
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[723  22]
 [ 27  62]]
classification report
              precision    recall  f1-score   support

           0       0

Selected Feature 8
Index(['melspectrogram_max_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'max perc_6dB', 'max perc_0dB',
       'min perc_0dB', 'max perc_-6dB', 'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[712  22]
 [ 26  74]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       734
           1       0.77      0.74      0.76       100

    accuracy                           0.94       834
   macro avg       0.87      0.86      0.86       834
weighted avg       0.94      0.94      0.94       834

accuracy score
0.9424460431654677
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[729  16]
 [ 30  59]]
classification report
              precision  

Selected Feature 10
Index(['melspectrogram_max_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'max perc_6dB', 'zero_crossing_rate_0dB',
       'max perc_0dB', 'min harm_0dB', 'min perc_0dB', 'max perc_-6dB',
       'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[714  20]
 [ 19  81]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       734
           1       0.80      0.81      0.81       100

    accuracy                           0.95       834
   macro avg       0.89      0.89      0.89       834
weighted avg       0.95      0.95      0.95       834

accuracy score
0.9532374100719424
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[723  22]
 [ 26  63

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[726   8]
 [ 20  80]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       734
           1       0.91      0.80      0.85       100

    accuracy                           0.97       834
   macro avg       0.94      0.89      0.92       834
weighted avg       0.97      0.97      0.97       834

accuracy score
0.9664268585131894
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[741   4]
 [ 32  57]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       745
           1       0.93      0.64      0.76        89

    accuracy                           

...

Selected Feature 15
Index(['melspectrogram_max_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'max perc_6dB', 'min harm_6dB',
       'min perc_6dB', 'melspectrogram_sum_0dB', 'melspectrogram_corr_0dB',
       'melspectrogram_std_0dB', 'zero_crossing_rate_0dB', 'max perc_0dB',
       'min harm_0dB', 'min perc_0dB', 'max perc_-6dB', 'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[714  20]
 [ 29  71]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       734
           1       0.78      0.71      0.74       100

    accuracy                           0.94       834
   macro avg       0.87      0.84      0.86       834
weighted avg       0.94      0.94      0.94       834

accuracy score
0.9412470023980816
----------------------------------------------
TRAIN-VA

Selected Feature 16
Index(['melspectrogram_max_6dB', 'melspectrogram_sum_6dB',
       'melspectrogram_corr_6dB', 'melspectrogram_std_6dB', 'rms_6dB',
       'max harm_6dB', 'max perc_6dB', 'min harm_6dB', 'min perc_6dB',
       'melspectrogram_max_0dB', 'melspectrogram_corr_0dB', 'max perc_0dB',
       'min perc_0dB', 'melspectrogram_corr_-6dB', 'max perc_-6dB',
       'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[728   6]
 [ 29  71]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       734
           1       0.92      0.71      0.80       100

    accuracy                           0.96       834
   macro avg       0.94      0.85      0.89       834
weighted avg       0.96      0.96      0.96       834

accuracy score
0.9580335731414868
------------

Selected Feature 18
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_corr_6dB', 'melspectrogram_std_6dB', 'mfcc_6dB',
       'rms_6dB', 'max perc_6dB', 'min harm_6dB', 'min perc_6dB',
       'melspectrogram_0dB', 'melspectrogram_max_0dB',
       'melspectrogram_corr_0dB', 'melspectrogram_std_0dB', 'max perc_0dB',
       'min perc_0dB', 'melspectrogram_corr_-6dB', 'max perc_-6dB',
       'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[730   4]
 [ 24  76]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       734
           1       0.95      0.76      0.84       100

    accuracy                           0.97       834
   macro avg       0.96      0.88      0.91       834
weighted avg       0.97      0.97      0.96       834

accuracy score
0.96642

Selected Feature 20
Index(['melspectrogram_max_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'max perc_6dB', 'min harm_6dB',
       'min perc_6dB', 'melspectrogram_max_0dB', 'melspectrogram_sum_0dB',
       'melspectrogram_corr_0dB', 'melspectrogram_std_0dB', 'mfcc_0dB',
       'zero_crossing_rate_0dB', 'mean harm_0dB', 'max perc_0dB',
       'min harm_0dB', 'min perc_0dB', 'melspectrogram_max_-6dB',
       'melspectrogram_corr_-6dB', 'max perc_-6dB', 'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[716  18]
 [ 25  75]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       734
           1       0.81      0.75      0.78       100

    accuracy                           0.95       834
   macro avg       0.89      0.86      0.87       834
weighted avg       0

Selected Feature 21
Index(['melspectrogram_6dB', 'melspectrogram_max_6dB',
       'melspectrogram_sum_6dB', 'melspectrogram_corr_6dB',
       'melspectrogram_std_6dB', 'mfcc_6dB', 'rms_6dB',
       'spectral_bandwidth_6dB', 'spectral_flatness _6dB', 'max harm_6dB',
       'max perc_6dB', 'min harm_6dB', 'min perc_6dB', 'melspectrogram_0dB',
       'melspectrogram_max_0dB', 'melspectrogram_corr_0dB', 'max perc_0dB',
       'min perc_0dB', 'melspectrogram_corr_-6dB', 'max perc_-6dB',
       'min perc_-6dB'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[730   4]
 [ 32  68]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       734
           1       0.94      0.68      0.79       100

    accuracy                           0.96       834
   macro avg       0.95      0.84